# Model Definition

The following code defines the models to be evaluated (according to the best configurations found during testing), these were then saved to IBM COS for loading later.

In [20]:
import numpy as np
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils import to_categorical, plot_model
from keras import backend as K
from time import time
from collections import Counter
import matplotlib.pyplot as plt
import os
%matplotlib inline
from sklearn.multiclass import OneVsOneClassifier
from sklearn import svm, preprocessing
from sklearn.metrics import classification_report, confusion_matrix  
import pickle

## SVM Definition

In [22]:
C=10

#svm_mod = svm.LinearSVC(C=C, max_iter=10)
svm_mod = OneVsOneClassifier(svm.LinearSVC(random_state=0, C=C, max_iter=10))

In [23]:
filename = './models/svm.sav'
pickle.dump(svm_mod, open(filename, 'wb'))

## Neural Network MLP Definition

In [ ]:
nn_mod = MLPClassifier(hidden_layer_sizes = (50,))

In [ ]:
filename = './models/Neural Network.sav'
pickle.dump(nn_mod, open(filename, 'wb'))

## Deep CNN (RESNET) Definition

In [12]:
from keras.layers import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.merge import Add
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras.utils import plot_model


#%%
class ResNet():
    """
    Usage: 
        sr = ResNet([4,8,16], input_size=(50,50,1), output_size=12)
        sr.build()
        followed by sr.m.compile(loss='categorical_crossentropy', 
                                 optimizer='adadelta', metrics=["accuracy"])
        save plotted model with: 
            keras.utils.plot_model(sr.m, to_file = '<location>.png', 
                                   show_shapes=True)
    """
    def __init__(self,
                 filters_list=[], 
                 input_size=None, 
                 output_size=None,
                 initializer='glorot_uniform'):
        self.filters_list = filters_list
        self.input_size = input_size
        self.output_size = output_size
        self.initializer = initializer
        self.m = None        
    
    def _block(self, filters, inp):
        """ one residual block in a ResNet
        
        Args:
            filters (int): number of convolutional filters
            inp (tf.tensor): output from previous layer
            
        Returns:
            tf.tensor: output of residual block
        """
        layer_1 = BatchNormalization()(inp)
        act_1 = Activation('relu')(layer_1)
        conv_1 = Conv2D(filters, (3,3), 
                        padding = 'same', 
                        kernel_initializer = self.initializer)(act_1)
        layer_2 = BatchNormalization()(conv_1)
        act_2 = Activation('relu')(layer_2)
        conv_2 = Conv2D(filters, (3,3), 
                        padding = 'same', 
                        kernel_initializer = self.initializer)(act_2)
        return(conv_2)

    def build(self):
        """
        Returns:
            keras.engine.training.Model
        """
        i = Input(shape = self.input_size, name = 'input')
        x = Conv2D(self.filters_list[0], (3,3), 
                   padding = 'same', 
                   kernel_initializer = self.initializer)(i)
        x = MaxPooling2D(padding = 'same')(x)        
        x = Add()([self._block(self.filters_list[0], x),x])
        x = Add()([self._block(self.filters_list[0], x),x])
        x = Add()([self._block(self.filters_list[0], x),x])
        if len(self.filters_list) > 1:
            for filt in self.filters_list[1:]:
                x = Conv2D(filt, (3,3),
                           strides = (2,2),
                           padding = 'same',
                           activation = 'relu',
                           kernel_initializer = self.initializer)(x)
                x = Add()([self._block(filt, x),x])
                x = Add()([self._block(filt, x),x])
                x = Add()([self._block(filt, x),x])
        x = GlobalAveragePooling2D()(x)
        x = Dense(self.output_size, activation = 'softmax')(x)
        
        self.m = Model(i,x)
        return self.m

In [25]:
X_train = np.load('./data/X_train.npy')
filters_list = [8,16,32]
output_size = 12
input_size = X_train.shape[1:]

In [16]:
sr = ResNet(filters_list, input_size, output_size)
sr.build()
sr.m.compile(loss='categorical_crossentropy', 
             optimizer='adadelta', 
             metrics=['accuracy'])

In [18]:
filename = './models/RESNET.h5'
sr.m.save(filename)

save all models to IBM cos

In [32]:
# The code was removed by Watson Studio for sharing.

In [27]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

In [30]:

cos.upload_file(Filename='models/RESNET.h5',Bucket=credentials_1['BUCKET'],Key='RESNET.h5')
cos.upload_file(Filename='models/Neural Network.sav',Bucket=credentials_1['BUCKET'],Key='Neural Network.sav')
cos.upload_file(Filename='models/svm.sav',Bucket=credentials_1['BUCKET'],Key='svm.sav')